In [1]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

True

In [2]:
formato_numerico = "{0:10.2f}"
ativo = list(())
AtivoSymbolInfo = list(())
matriz = list(())
global matriz

In [3]:
def imprimirInfoAtivo(t, y, n):
    print(t, y, formato_numerico.format(n))

In [4]:
# ****************************************************
def RolarOpcoes(ativo):
    n = len(ativo)
    AtivoSymbolInfo.clear()
    rows,cols = (n,10)
    matriz = [[0 for i in range(cols)] for j in range(rows)]
    
    for i in range(n):
        AtivoSymbolInfo.append(mt5.symbol_info(ativo[i]))
        xx = AtivoSymbolInfo[i]
        matriz[i][0] = xx.name
        matriz[i][1] = xx.option_strike
        matriz[i][4] = xx.last
 
        # somente para linha 1, que é da opção vendida, que preciso rolar
        # pega o ASK... mas se não tiver ninguém oferecendo, pega o LAST
        if i == 1:
            matriz[1][3] = xx.ask
            if matriz[1][3] == 0:
                fff = input('Entre o valor do Ask ' + xx.name + ': ')
                matriz[i][3] = float(fff)
            gasto = matriz[1][3] * -1
#            if matriz[1][3] == 0:       gasto = matriz[1][4]
 
        # para as linhas 2,3,... ou seja, para as opções dos meses seguintes que poderei vender
        if i>1:
            matriz[i][2] = xx.bid
            if matriz[i][2] == 0:
                fff = input('Entre o valor do Bid ' + xx.name + ': ')
                matriz[i][2] = float(fff)
            matriz[i][5] = matriz[i][1] - matriz[1][1] # aumento em R$ strike nova - strike vendida
            matriz[i][6] = (matriz[i][1]/matriz[1][1] - 1) * 100  # % strike nova - strike vendido
            matriz[i][7] = matriz[i][5] + gasto + matriz[i][2] # dif.strike - gasto
            matriz[i][8] = matriz[i][7] / matriz[1][1] * 100
            matriz[i][9] = matriz[i][2] - matriz[1][3] 

    df = pd.DataFrame(matriz, columns=['Ativo','Strike','bid','ask','Last','▲ R$','▲ % Strike','Somatório','% sobre '+matriz[1][0], 'Cash'])
    print(df)
    nome_arq = 'rolar_opcoes_' + ativo[0] + '_' + ativo[1] + '_' + ativo[2]+ '.xlsx'
    df.to_excel(nome_arq)

In [5]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH139'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid  ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.0  12.66  0.00    0.000000       0.00   
1  CMIGG154   13.35  0.00  0.2   0.19  0.00    0.000000       0.00   
2  CMIGH139   14.00  0.28  0.0   0.28  0.65    4.868914       0.73   

   % sobre CMIGG154  Cash  
0          0.000000  0.00  
1          0.000000  0.00  
2          5.468165  0.08  


In [6]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH134'] 
RolarOpcoes(ativo)

      Ativo  Strike  bid  ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.0  0.0  12.65  0.00    0.000000       0.00   
1  CMIGG154   13.35  0.0  0.2   0.19  0.00    0.000000       0.00   
2  CMIGH134   13.50  0.4  0.0   0.50  0.15    1.123596       0.35   

   % sobre CMIGG154  Cash  
0          0.000000   0.0  
1          0.000000   0.0  
2          2.621723   0.2  


In [7]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH131', 'CMIGH134', 'CMIGH139'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid  ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.0  12.65  0.00    0.000000       0.00   
1  CMIGG154   13.35  0.00  0.2   0.19  0.00    0.000000       0.00   
2  CMIGH131   13.25  0.48  0.0   0.55 -0.10   -0.749064       0.18   
3  CMIGH134   13.50  0.40  0.0   0.50  0.15    1.123596       0.35   
4  CMIGH139   14.00  0.28  0.0   0.28  0.65    4.868914       0.73   

   % sobre CMIGG154  Cash  
0          0.000000  0.00  
1          0.000000  0.00  
2          1.348315  0.28  
3          2.621723  0.20  
4          5.468165  0.08  


In [8]:
ativo = ['ELET3', 'ELETG454', 'ELETH460', 'ELETH467', 'ELETH470', 'ELETH477', 'ELETH480'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask  Last  ▲ R$  ▲ % Strike  Somatório  \
0     ELET3    0.00  0.00  0.00  44.8  0.00    0.000000       0.00   
1  ELETG454   45.61  0.00  1.18   0.0  0.00    0.000000       0.00   
2  ELETH460   46.00  4.11  0.00   0.0  0.39    0.855076       3.32   
3  ELETH467   46.75  1.79  0.00   0.0  1.14    2.499452       1.75   
4  ELETH470   47.00  1.76  0.00   0.0  1.39    3.047577       1.97   
5  ELETH477   47.75  1.45  0.00   0.0  2.14    4.691954       2.41   
6  ELETH480   48.00  1.42  0.00   0.0  2.39    5.240079       2.63   

   % sobre ELETG454  Cash  
0          0.000000  0.00  
1          0.000000  0.00  
2          7.279105  2.93  
3          3.836878  0.61  
4          4.319228  0.58  
5          5.283929  0.27  
6          5.766279  0.24  


In [16]:
ativo = ['ITSA4', 'ITSAG113', 'ITSAH118'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     ITSA4    0.00  0.00  0.00  11.54  0.00    0.000000       0.00   
1  ITSAG113   11.26  0.00  0.51   0.51  0.00    0.000000       0.00   
2  ITSAH118   11.79  0.43  0.00   0.43  0.53    4.706927       0.45   

   % sobre ITSAG113  Cash  
0          0.000000  0.00  
1          0.000000  0.00  
2          3.996448 -0.08  


In [10]:
ativo = ['CSAN3', 'CSANG100', 'CSANH250'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CSAN3    0.00  0.00  0.00  24.94  0.00     0.00000       0.00   
1  CSANG100   24.74  0.00  1.03   0.89  0.00     0.00000       0.00   
2  CSANH250   25.00  1.42  0.00   1.50  0.26     1.05093       0.65   

   % sobre CSANG100  Cash  
0          0.000000  0.00  
1          0.000000  0.00  
2          2.627324  0.39  


In [11]:
ativo = ['BBSE3', 'BBSET220', 'BBSET230'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid  ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     BBSE3     0.0  0.00  0.0  24.08   0.0    0.000000       0.00   
1  BBSET220    22.0  0.00  0.2   0.18   0.0    0.000000       0.00   
2  BBSET230    23.0  0.37  0.0   0.39   1.0    4.545455       1.17   

   % sobre BBSET220  Cash  
0          0.000000  0.00  
1          0.000000  0.00  
2          5.318182  0.17  
